# Santa 2025 - Quick EDA

In [1]:
import pandas as pd
import numpy as np

# Load sample submission
df = pd.read_csv('/home/data/sample_submission.csv')
print(f'Shape: {df.shape}')
print(f'Columns: {df.columns.tolist()}')
df.head(20)

Shape: (20100, 4)
Columns: ['id', 'x', 'y', 'deg']


,id,x,y,deg
0,001_0,s0.0,s0.0,s90.0
1,002_0,s0.0,s0.0,s90.0
2,002_1,s0.202736,s-0.511271,s90.0
3,003_0,s0.0,s0.0,s90.0
4,003_1,s0.202736,s-0.511271,s90.0
5,003_2,s0.5206,s0.177413,s180.0
6,004_0,s0.0,s0.0,s90.0
7,004_1,s0.202736,s-0.511271,s90.0
8,004_2,s0.5206,s0.177413,s180.0
9,004_3,s-0.818657,s-0.228694,s180.0


In [2]:
# Check the pre-optimized solution
import os

preopt_path = '/home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/santa-2025.csv'
if os.path.exists(preopt_path):
    df_preopt = pd.read_csv(preopt_path)
    print(f'Pre-optimized shape: {df_preopt.shape}')
    print(df_preopt.head(10))
else:
    print('Pre-optimized file not found')

Pre-optimized shape: (20100, 4)
      id                       x                       y  \
0  001_0    s-48.196086194214246     s58.770984615214225   
1  002_0   s0.154097069621355887  s-0.038540742694794648   
2  002_1  s-0.154097069621372845  s-0.561459257305224058   
3  003_0      s1.123655816140301      s0.781101815992563   
4  003_1       s1.23405569584216      s1.275999500663759   
5  003_2      s0.641714640229075      s1.180458566613381   
6  004_0  s-0.324747789589372171   s0.132109978088185392   
7  004_1   s0.315354346242637695   s0.132109978063475492   
8  004_2   s0.324747789592379210  s-0.732109978069475531   
9  004_3  s-0.315354348134818330  s-0.732109978094185987   

                       deg  
0                    s45.0  
1  s203.629377730656841550  
2   s23.629377730656791812  
3        s111.125132292893  
4         s66.370622269343  
5      s155.13405193710082  
6  s156.370622145636389178  
7  s156.370622269264089255  
8  s336.370622269264003990  
9  s336.370622145

In [3]:
# Calculate score for pre-optimized solution
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.ops import unary_union

getcontext().prec = 25

# Tree geometry
TX = [0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125]
TY = [0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5]

def get_tree_polygon(x, y, deg):
    """Create tree polygon at position (x, y) with rotation deg"""
    poly = Polygon(zip(TX, TY))
    poly = affinity.rotate(poly, deg, origin=(0, 0))
    poly = affinity.translate(poly, x, y)
    return poly

def calculate_score(df):
    """Calculate total score for a submission"""
    # Parse values (remove 's' prefix)
    df = df.copy()
    df['x_val'] = df['x'].str[1:].astype(float)
    df['y_val'] = df['y'].str[1:].astype(float)
    df['deg_val'] = df['deg'].str[1:].astype(float)
    df['n'] = df['id'].str.split('_').str[0].astype(int)
    
    total_score = 0
    for n in range(1, 201):
        group = df[df['n'] == n]
        if len(group) == 0:
            continue
        
        # Get all tree polygons
        polys = [get_tree_polygon(row['x_val'], row['y_val'], row['deg_val']) 
                 for _, row in group.iterrows()]
        
        # Calculate bounding box
        union = unary_union(polys)
        bounds = union.bounds  # (minx, miny, maxx, maxy)
        side = max(bounds[2] - bounds[0], bounds[3] - bounds[1])
        
        # Add to score
        total_score += (side ** 2) / n
    
    return total_score

# Calculate score for pre-optimized
score = calculate_score(df_preopt)
print(f'Pre-optimized score: {score:.6f}')

Pre-optimized score: 70.676102
